# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [2]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.04500000085681677
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: 0.04500000085681677
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291


When finished, you can close the environment.

In [6]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

#### Imports

In [2]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from unityagents import UnityEnvironment
import numpy as np

In [4]:
from ounoise import OUNoise
from model import Actor, Critic
from replay_buffer import ReplayBuffer
from utils import plot_scores

#### Environment Setup

In [5]:
env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [6]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
BUFFER_SIZE = 100000  # replay buffer size 10^6 (paper)
BATCH_SIZE = 64       # minibatch size 64 (paper)
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 1e-4 (paper)
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0

#### Create Agent

In [9]:
from agent import Agent

def create_agent(env, brain_name, device, actor_file=None, critic_file=None, random_seed=39, fc1=128, fc2=128,
                 lr_actor=1e-04, lr_critic=1e-04, weight_decay=0, buffer_size=100000,
                 batch_size=64, gamma=0.99, tau=1e-3):
    brain = env.brains[brain_name]
    env_info = env.reset(train_mode=True)[brain_name]

    # size of each action
    action_size = brain.vector_action_space_size
    states = env_info.vector_observations
    state_size = states.shape[1]

    agent = Agent(device, state_size=state_size, action_size=action_size, random_seed=random_seed, fc1=fc1, fc2=fc2,
                  lr_actor=lr_actor, lr_critic=lr_critic, weight_decay=weight_decay, buffer_size=buffer_size,
                  batch_size=batch_size, gamma=gamma, tau=tau)

    if actor_file and critic_file:
        agent.actor_local.load_state_dict(torch.load(actor_file))
        agent.critic_local.load_state_dict(torch.load(critic_file))

    return agent

In [10]:
agent1 = create_agent(env, brain_name, device, actor_file=None, critic_file=None, random_seed=39, lr_actor=LR_ACTOR,
                           lr_critic=LR_CRITIC, weight_decay=WEIGHT_DECAY, buffer_size=BUFFER_SIZE,
                           batch_size=BATCH_SIZE, gamma=GAMMA, tau=TAU)
agent2 = create_agent(env, brain_name, device, actor_file=None, critic_file=None, random_seed=39, lr_actor=LR_ACTOR,
                           lr_critic=LR_CRITIC, weight_decay=WEIGHT_DECAY, buffer_size=BUFFER_SIZE,
                           batch_size=BATCH_SIZE, gamma=GAMMA, tau=TAU)

Initilized Actor with state size 24 and action size 2
Initilized Actor with state size 24 and action size 2
Initilized Critic with state size 24 and action size 2
Initilized Critic with state size 24 and action size 2
Initilized agent with state size = 24 and action size = 2
Initilized Actor with state size 24 and action size 2
Initilized Actor with state size 24 and action size 2
Initilized Critic with state size 24 and action size 2
Initilized Critic with state size 24 and action size 2
Initilized agent with state size = 24 and action size = 2


#### Training

In [11]:
def ddpg(n_episodes=2500, print_every=100):
    agent1_reward = []
    agent2_reward = []

    for i_episode in range(1, n_episodes + 1):
        # Reset environment
        env_info = env.reset(train_mode=True)[brain_name]
        # Reset agent
        agent1.reset()
        agent2.reset()
        # Get the initial state
        states = env_info.vector_observations

        episode_reward = np.array([0., 0.])

        while True:
            # Get action
            action_agent1 = agent1.act(states[0])
            action_agent2 = agent2.act(states[1])
            actions = np.array([action_agent1[0], action_agent2[0]])

            # Observe reaction (environment)
            env_info = env.step(actions)[brain_name]
            ## Get new state
            next_states = env_info.vector_observations
            ## Get reward
            rewards = env_info.rewards
            # See if episode has finished
            dones = env_info.local_done
            # Step
            agent1.step(states[0], action_agent1, rewards[0], next_states[0], dones[0])
            agent2.step(states[1], action_agent2, rewards[1], next_states[1], dones[1])

            states = next_states
            episode_reward += rewards

            if np.any(dones):
                break

        agent1_reward.append(episode_reward[0])
        agent2_reward.append(episode_reward[1])

        if i_episode % print_every == 0:
            avg_rewards = [np.mean(agent1_reward[-100:]), np.mean(agent2_reward[-100:])]
            print("\rEpisode {} - \tAverage Score: {:.2f} {:.2f}".format(i_episode, avg_rewards[0], avg_rewards[1]),
                  end="")

            torch.save(agent1.actor_local.state_dict(), 'agent1_actor_checkpoint.pth')
            torch.save(agent1.critic_local.state_dict(), 'agent1_critic_checkpoint.pth')

            torch.save(agent2.actor_local.state_dict(), 'agent2_actor_checkpoint.pth')
            torch.save(agent2.critic_local.state_dict(), 'agent2_critic_checkpoint.pth')

    return {'agent1_scores': agent1_reward, 'agent2_scores': agent2_reward}

In [ ]:
scores = ddpg()

Episode 2300 - 	Average Score: 1.03 1.041

In [9]:
env.close()

NameError: name 'env' is not defined

#### Scores

In [10]:
plot_scores(scores['agent1_scores'])

NameError: name 'plot_scores' is not defined

In [11]:
plot_scores(scores['agent2_scores'])

NameError: name 'plot_scores' is not defined

In [2]:
max_scores = [max(scores['agent1_scores'][i], scores['agent2_scores'][i]) for i in range(len(scores['agent1_scores']))]

NameError: name 'scores' is not defined

In [3]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from scipy.interpolate import spline
%matplotlib inline

In [4]:
from unityagents import UnityEnvironment
import numpy as np

In [5]:
env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [6]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
BUFFER_SIZE = 100000  # replay buffer size 10^6 (paper)
BATCH_SIZE = 64       # minibatch size 64 (paper)
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 1e-4 (paper)
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0

In [12]:
from agent import Agent

def create_agent(env, brain_name, device, actor_file=None, critic_file=None, random_seed=39, fc1=128, fc2=128,
                 lr_actor=1e-04, lr_critic=1e-04, weight_decay=0, buffer_size=100000,
                 batch_size=64, gamma=0.99, tau=1e-3):
    brain = env.brains[brain_name]
    env_info = env.reset(train_mode=True)[brain_name]

    # size of each action
    action_size = brain.vector_action_space_size
    states = env_info.vector_observations
    state_size = states.shape[1]

    agent = Agent(device, state_size=state_size, action_size=action_size, random_seed=random_seed, fc1=fc1, fc2=fc2,
                  lr_actor=lr_actor, lr_critic=lr_critic, weight_decay=weight_decay, buffer_size=buffer_size,
                  batch_size=batch_size, gamma=gamma, tau=tau)

    if actor_file and critic_file:
        agent.actor_local.load_state_dict(torch.load(actor_file))
        agent.critic_local.load_state_dict(torch.load(critic_file))

    return agent

In [13]:
agent1 = create_agent(env, brain_name, device, actor_file='agent1_actor_checkpoint.pth', 
                           critic_file='agent1_critic_checkpoint.pth', 
                           random_seed=39, lr_actor=LR_ACTOR, lr_critic=LR_CRITIC, weight_decay=WEIGHT_DECAY, 
                           buffer_size=BUFFER_SIZE, batch_size=BATCH_SIZE, gamma=GAMMA, tau=TAU)
agent2 = create_agent(env, brain_name, device, actor_file='agent2_actor_checkpoint.pth', 
                           critic_file='agent2_critic_checkpoint.pth',
                           random_seed=39, lr_actor=LR_ACTOR, lr_critic=LR_CRITIC, weight_decay=WEIGHT_DECAY, 
                           buffer_size=BUFFER_SIZE, batch_size=BATCH_SIZE, gamma=GAMMA, tau=TAU)

Initilized Actor with state size 24 and action size 2
Initilized Actor with state size 24 and action size 2
Initilized Critic with state size 24 and action size 2
Initilized Critic with state size 24 and action size 2
Initilized agent with state size = 24 and action size = 2


RuntimeError: cuda runtime error (35) : CUDA driver version is insufficient for CUDA runtime version at torch/csrc/cuda/Module.cpp:51